In [2]:
import pandas as pd
import numpy as np
import math
import statistics
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
kcdf = pd.read_csv('~/Documents/GitHub/regression-washington/data/kc_house_data.csv')
train_data = pd.read_csv('~/Documents/GitHub/regression-washington/data/kc_house_train_data.csv')
test_data = pd.read_csv('~/Documents/GitHub/regression-washington/data/kc_house_test_data.csv')

In [13]:
# Step 3
train_data['bedrooms_squared'] =  train_data['bedrooms']**2
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
train_data['log_sqft_living'] = np.log(train_data['sqft_living'])
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['bedrooms_squared'] =  test_data['bedrooms']**2
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']
test_data['log_sqft_living'] = np.log(test_data['sqft_living'])
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [5]:
# 4. Quiz Question: what are the mean (arithmetic average) values of your 4 new variables on TEST data? 
#(round to 2 digits)
bedrooms_squared_mean = test_data['bedrooms_squared'].mean(); print("bedrooms_squared_mean:", round(bedrooms_squared_mean,2))
bed_bath_rooms_mean = test_data['bed_bath_rooms'].mean(); print("bed_bath_rooms_mean:", round(bed_bath_rooms_mean,2))
log_sqft_living_mean = test_data['log_sqft_living'].mean(); print("log_sqft_living_mean:", round(log_sqft_living_mean,2))
lat_plus_long_mean = test_data['lat_plus_long'].mean(); print("lat_plus_long_mean:", round(lat_plus_long_mean,2))

bedrooms_squared_mean: 12.45
bed_bath_rooms_mean: 7.5
log_sqft_living_mean: 7.55
lat_plus_long_mean: -74.65


In [20]:
# Model 1: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’, and ‘long’

rg_model1 = linear_model.LinearRegression()
y_train = train_data['price']
model1_x = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']]
rg_model1.fit(model1_x, y_train)

#Model 2: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’,‘long’, and ‘bed_bath_rooms’
rg_model2 = linear_model.LinearRegression()
model2_x = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms']]
rg_model2.fit(model2_x, y_train)

#Model 3: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’,‘long’, ‘bed_bath_rooms’, 
#‘bedrooms_squared’, ‘log_sqft_living’, and ‘lat_plus_long’
rg_model3 = linear_model.LinearRegression()
model3_x = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms',
                       'bedrooms_squared','log_sqft_living','lat_plus_long']]
rg_model3.fit(model3_x, y_train)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [8]:
#6. Quiz Question: What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 1?

#7. Quiz Question: What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 2?
print('Coefficients model1: \n',list(zip(['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long'], rg_model1.coef_)))
print('Coefficients model2: \n', list(zip(['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms'], rg_model2.coef_)))
print('Coefficients model3: \n', list(zip(['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms',
                       'bedrooms_squared','log_sqft_living','lat_plus_long'], rg_model3.coef_)))

Coefficients model1: 
 [('sqft_living', 312.2586462732027), ('bedrooms', -59586.53315361222), ('bathrooms', 15706.742082734667), ('lat', 658619.2639305154), ('long', -309374.35126823327)]
Coefficients model2: 
 [('sqft_living', 306.6100534589955), ('bedrooms', -113446.36807020336), ('bathrooms', -71461.3082927587), ('lat', 654844.6295033012), ('long', -294298.9691381188), ('bed_bath_rooms', 25579.652000752052)]
Coefficients model3: 
 [('sqft_living', 529.4228196465226), ('bedrooms', 34514.229577989994), ('bathrooms', 67060.78131891116), ('lat', 534085.6108674947), ('long', -406750.7108610399), ('bed_bath_rooms', -8570.504394630101), ('bedrooms_squared', -6788.586670340511), ('log_sqft_living', -561831.4840755209), ('lat_plus_long', 127334.9000064558)]


In [9]:
print(((rg_model1.predict(model1_x) - y_train) ** 2).sum())
print(((rg_model2.predict(model2_x) - y_train) ** 2).sum())
print(((rg_model3.predict(model3_x) - y_train) ** 2).sum())



967879963049545.6
958419635074068.5
903436455050477.6


In [18]:
model1_x_test = test_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']]
model2_x_test = test_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms']]
model3_x_test = test_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms',
                           'bedrooms_squared', 'log_sqft_living','lat_plus_long']]
ytest = test_data['price']
print(((rg_model1.predict(model1_x_test) - ytest) ** 2).sum())
print(((rg_model2.predict(model2_x_test) - ytest) ** 2).sum())
print(((rg_model3.predict(model3_x_test) - ytest) ** 2).sum())

225500469795490.0
223377462976466.88
259236319207179.94


# Multiple Linear Regression Quiz 2
## Implementing gradient descent

In [38]:
kcdf = pd.read_csv('~/Documents/GitHub/regression-washington/data/kc_house_data.csv')
train_data = pd.read_csv('~/Documents/GitHub/regression-washington/data/kc_house_train_data.csv')
test_data = pd.read_csv('~/Documents/GitHub/regression-washington/data/kc_house_test_data.csv')

In [130]:
# Step 3
def get_numpy_data(data, features, output):
    data['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = data[features]
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’ 
    # this will convert the SArray into a numpy array:
    output_array = data[output] # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [73]:
a,b = get_numpy_data(test_data, ['sqft_living', 'bedrooms'], 'price')

In [50]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights) 
    return(predictions)

In [138]:
def feature_derivative(errors, feature):
    feature_matrix = np.array(feature)
    derivative = 2*np.dot(feature_matrix, errors) 
    return(derivative)

In [235]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    gradient_sum_squares = 0 
    output = np.array(output)
    counter = 0
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        # compute the errors as predictions - output:
        predictions =  predict_outcome(feature_matrix, weights)
        errors = output - predictions
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # print(feature_matrix.iloc[:,1])
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix.iloc[:,i])
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares += derivative **2
            # update the weight based on step size and derivative:
            #print(step_size * derivative) 
            weights[i] = weights[i] + (step_size * derivative)            
            gradient_magnitude = math.sqrt(gradient_sum_squares)
        #print(gradient_magnitude, tolerance)
        if (gradient_magnitude < tolerance or counter > 10000):
            converged = True
        counter +=1
    return(weights, counter)


In [255]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7


In [256]:
errors = output - predict_outcome(simple_feature_matrix, initial_weights)
np.dot(np.array(simple_feature_matrix.iloc[:,i]),np.array(errors))
#simple_feature_matrix.iloc[:,i]
#feature_derivative(errors, simple_feature_matrix.iloc[:,i])

10157238227.0

In [257]:
weights, iterations = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
print(weights, iterations)

[-46999.88716555    281.91211918] 12


In [221]:
#11. Quiz Question: What is the predicted price for the 1st house in the Test data set for model 1 
#(round to nearest dollar)?
(feature_matrix_test,output)  = get_numpy_data(test_data, ['sqft_living'], 'price')
round(predict_outcome(feature_matrix_test.iloc[0],weights))


356134.0

In [225]:
# 12. Now compute RSS on all test data for this model. Record the value and store it for later
RSS = sum((output - predict_outcome(feature_matrix_test,weights))**2)
RSS
#275400044902128.31


275400044902128.78

In [240]:
# 13. Now we will use the gradient descent to fit a model with more than 1 predictor variable (and an intercept). 
#Use the following parameters:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9
weights, iterations = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)
print(weights, iterations)

[-9.99999688e+04  2.45072603e+02  6.52795267e+01] 274


In [242]:
# 14. Use the regression weights from this second model 
# (using sqft_living and sqft_living_15) and predict the outcome of all the house prices on the TEST data.
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
predicted_house_prices_model2 = predict_outcome(test_feature_matrix, weights)


366651.0

In [243]:
# 15. Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 
# (round to nearest dollar)?
round(predicted_house_prices_model2[0])

366651.0

In [247]:
#17. Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?
print(output[0] - predicted_house_prices_model2[0])
print(output[0] - 356134.0)

-144751.41162949387
-134234.0


In [254]:
RSS2 = sum((test_output - predicted_house_prices_model2)**2)
print(" RSS model 1:", RSS,"\n","RSS model 2:", RSS2)


 RSS model 1: 275400044902128.78 
 RSS model 2: 270263443629803.3
